In [ ]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from google.api_core.exceptions import Conflict
from datetime import datetime

# 1️⃣ Authentication and Setup

PROJECT_ID = "trim-plexus-396409"
DATASET_ID = "Silver_Layer"
TABLE_ID = "Cleansed_Supply_Chain_Data"
CSV_PATH = r"E:\Supply Chain Data Integration System\Global_Superstore.csv"

key_path = r"E:\BigQueryAssignment\trim-plexus-396409-dfc55c39f51e.json"  # change this path
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# 2️⃣ Extract: Read CSV file into pandas
print("📥 Extracting data from CSV...")
df = pd.read_csv(CSV_PATH, encoding='latin1')

print("✅ Data extracted. Sample:")
print(df.head())

# 3️⃣ Transform: Clean and enrich data
print("🔄 Transforming data...")

# Example transformations:
# - Remove duplicates
# - Fill missing profits with median value of the 'Profit' column

# Deduplicate using Order ID as key (normalize first). If Order ID is missing, fall back to full-row dedupe.
if 'Order ID' in df.columns:
    # Normalize Order ID to string and strip whitespace
    df['Order ID'] = df['Order ID'].astype(str).str.strip()
    rows_before = len(df)
    df.drop_duplicates(subset=['Order ID'], keep='first', inplace=True)
    rows_after = len(df)
    print(f"🧹 Deduplicated by Order ID: removed {rows_before - rows_after} rows")
else:
    df.drop_duplicates(keep='first', inplace=True)
profit_median = df['Profit'].median()
df['Profit'].fillna(profit_median, inplace=True)
temp_post = '00000'
def _format_postal(x):
    # Return a string postal code; preserve existing ones and avoid float-like '0.0'
    if pd.isna(x):
        return temp_post
    s = str(x).strip()
    # Remove trailing .0 coming from floats (e.g. 12345.0 -> '12345')
    if s.endswith('.0'):
        s = s[:-2]
    return s

df['Postal Code'] = df['Postal Code'].apply(_format_postal)
# --- Postal code enrichment using external API (OpenStreetMap Nominatim) with local cache ---
# This block will:
# 1) load an existing city->postal CSV cache if present,
# 2) lookup missing cities via Nominatim (respectful rate limit),
# 3) save the expanded cache to disk for future runs,
# 4) apply mapped postal codes where found and preserve existing codes otherwise.
import os, time, csv, json
try:
    import requests
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'requests'])
    import requests

CACHE_CSV = r'E:\Supply Chain Data Integration System\city_postal_map.csv'
# Load existing cache if present
city_postal_map = {}
if os.path.exists(CACHE_CSV):
    try:
        with open(CACHE_CSV, newline='', encoding='utf-8') as fh:
            reader = csv.reader(fh)
            for row in reader:
                if not row:
                    continue
                city = row[0].strip()
                code = row[1].strip() if len(row) > 1 else ''
                city_postal_map[city] = code
        print(f"🔁 Loaded {len(city_postal_map)} cached city->postal mappings from {CACHE_CSV}")
    except Exception as e:
        print('Could not read cache CSV:', e)
#-----------------------------------------------------------#
# Determine which cities to lookup
cities_in_df = sorted(df['City'].dropna().astype(str).str.strip().unique()) if 'City' in df.columns else []
to_lookup = [c for c in cities_in_df if c and (c not in city_postal_map or not city_postal_map.get(c))]
print(f"🔎 {len(to_lookup)} cities to lookup via Nominatim (cache misses).")

# Nominatim usage: be polite, include a descriptive User-Agent and sleep between requests
NOMINATIM_URL = 'https://nominatim.openstreetmap.org/search'
HEADERS = {'User-Agent': 'supply-chain-cleaner/1.0 (contact: your-email@example.com)'}
for city in to_lookup:
    try:
        params = {'q': city, 'format': 'json', 'addressdetails': 1, 'limit': 1}
        resp = requests.get(NOMINATIM_URL, params=params, headers=HEADERS, timeout=15)
        if resp.ok:
            data = resp.json()
            if data:
                addr = data[0].get('address', {})
                postcode = addr.get('postcode') or ''
                city_postal_map[city] = postcode
                print(f"  ↳ {city} -> {postcode}")
            else:
                city_postal_map[city] = ''
                print(f"  ↳ {city} -> (no result)")
        else:
            print(f"  ! HTTP {resp.status_code} for {city}")
            city_postal_map[city] = ''
    except Exception as e:
        print('  ! lookup error for', city, e)
        city_postal_map[city] = ''
    # Respect Nominatim usage policy: do not flood the service
    time.sleep(1)

# Persist cache back to CSV
try:
    with open(CACHE_CSV, 'w', newline='', encoding='utf-8') as fh:
        writer = csv.writer(fh)
        for k,v in sorted(city_postal_map.items()):
            writer.writerow([k, v])
    print(f"💾 Saved {len(city_postal_map)} city->postal mappings to {CACHE_CSV}")
except Exception as e:
    print('Could not save cache CSV:', e)

# Apply mapping to dataframe: where mapping has a non-empty code, use it; else keep existing Postal Code
if 'City' in df.columns:
    mapped = df['City'].astype(str).str.strip().map(city_postal_map)
    assigned_mask = mapped.notna() & (mapped != '')
    n_assigned = int(assigned_mask.sum())
    if n_assigned:
        print(f"📍 Assigned postal codes from API cache for {n_assigned} rows based on City.")
    df['Postal Code'] = mapped.where(assigned_mask, df.get('Postal Code'))
# Ensure Postal Code has fallback and is string-clean
df['Postal Code'] = df['Postal Code'].fillna(temp_post).astype(str).str.strip()
df['Postal Code'] = df['Postal Code'].str.replace(r'\.0$', '', regex=True)
#------------------------------------------------------------#

# Add new column with current load timestamp
df["load_timestamp"] = datetime.now()
# Parse dates explicitly as dd-mm-yyyy to avoid ambiguous parsing
if 'Order Date' in df.columns and 'Ship Date' in df.columns:
    df['_order_date_parsed'] = pd.to_datetime(df['Order Date'], format='%d-%m-%Y', errors='coerce')
    df['_ship_date_parsed'] = pd.to_datetime(df['Ship Date'], format='%d-%m-%Y', errors='coerce')
    n_order_unparsed = df['_order_date_parsed'].isna().sum()
    n_ship_unparsed = df['_ship_date_parsed'].isna().sum()
    if n_order_unparsed or n_ship_unparsed:
        print(f"⚠️ Parsing: {n_order_unparsed} 'Order Date' and {n_ship_unparsed} 'Ship Date' rows could not be parsed with dd-mm-yyyy format. These will have NaN durations.")
    # Compute durations
    df['Shipping_Duration_Days'] = (df['_ship_date_parsed'] - df['_order_date_parsed']).dt.days
    # Diagnostic: if durations are all zero (or suspicious), print samples to inspect parsing
    non_null = df['Shipping_Duration_Days'].dropna()
    if len(non_null) > 0 and (non_null.nunique() == 1 and non_null.iloc[0] == 0):
        print("⚠️ All non-null Shipping_Duration_Days are 0 — showing samples to diagnose parse issues:")
        print(df[['Order Date','Ship Date','_order_date_parsed','_ship_date_parsed','Shipping_Duration_Days']].head(10))
        # Try an alternate parsing strategy for inspection: month-first (mm-dd-yyyy) and inference
        df['_order_alt'] = pd.to_datetime(df['Order Date'], errors='coerce', dayfirst=False, infer_datetime_format=True)
        df['_ship_alt'] = pd.to_datetime(df['Ship Date'], errors='coerce', dayfirst=False, infer_datetime_format=True)
        print('Sample alternate parsed (dayfirst=False/infer):')
        print(df[['_order_alt','_ship_alt']].head(5))
    # drop helper parsed columns (keep if you'd like to audit)
    df.drop(columns=['_order_date_parsed','_ship_date_parsed'], inplace=True)
else:
    print("⚠️ 'Order Date' or 'Ship Date' column missing; setting 'Shipping_Duration_Days' to NA.")
    df['Shipping_Duration_Days'] = pd.NA

print("✅ Transformation complete. Sample:")
print(df.head())

# Save the cleansed data to a new CSV file
print("💾 Saving cleansed data to new CSV file...")
CLEANSED_CSV_PATH = r"E:\Supply Chain Data Integration System\Cleansed_Global_Superstore.csv"
df.to_csv(CLEANSED_CSV_PATH, index=False)
print(f"✅ Cleansed data saved to '{CLEANSED_CSV_PATH}'.")
# -------------------------


# 4️⃣ Load: Create dataset and table in BigQuery if not exists
dataset_ref = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")

try:
    client.create_dataset(dataset_ref)
    print(f"✅ Dataset '{DATASET_ID}' created.")
except Conflict:
    print(f"ℹ️ Dataset '{DATASET_ID}' already exists.")

table_ref = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"
table = bigquery.Table(table_ref)

try:
    client.create_table(table)
    print(f"✅ Table '{TABLE_ID}' created.")
except Conflict:
    print(f"ℹ️ Table '{TABLE_ID}' already exists.")

# 5️⃣ Load transformed data into BigQuery
print("🚀 Loading transformed data into BigQuery...")

job_config = bigquery.LoadJobConfig(
    autodetect=True,
    # Specify the source format. Common formats include CSV, NEWLINE_DELIMITED_JSON, AVRO.
    source_format=bigquery.SourceFormat.CSV,
    write_disposition="WRITE_TRUNCATE"  # Replace table data
)

job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

print(f"✅ Successfully loaded {len(df)} rows into {table_ref}.")


📥 Extracting data from CSV...
✅ Data extracted. Sample:
          Order ID  Order Date   Ship Date     Ship Mode Customer ID  \
0   CA-2012-124891  31-07-2012  31-07-2012      Same Day    RH-19495   
1    IN-2013-77878  05-02-2013  07-02-2013  Second Class    JR-16210   
2    IN-2013-71249  17-10-2013  18-10-2013   First Class    CR-12730   
3  ES-2013-1579342  28-01-2013  30-01-2013   First Class    KM-16375   
4     SG-2013-4320  05-11-2013  06-11-2013      Same Day     RH-9495   

      Customer Name      Segment           City            State  \
0       Rick Hansen     Consumer  New York City         New York   
1     Justin Ritter    Corporate     Wollongong  New South Wales   
2      Craig Reiter     Consumer       Brisbane       Queensland   
3  Katherine Murray  Home Office         Berlin           Berlin   
4       Rick Hansen     Consumer          Dakar            Dakar   

         Country  ...        Product ID    Category Sub-Category  \
0  United States  ...   TEC-AC-100

C:\Users\heman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\google\cloud\bigquery\_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


✅ Successfully loaded 25035 rows into trim-plexus-396409.Silver_Layer.Cleansed_Supply_Chain_Data.
